In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
#model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('train.csv', encoding = 'utf-8-sig')


In [4]:
df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

grouped = df.groupby("공종(중분류)")
# 결과 저장을 위한 딕셔너리
avg_similarities = {}

# 각 그룹에 대해 평균 코사인 유사도 계산
for category, group in grouped:
    sentences = group['재발방지대책 및 향후조치계획'].tolist()
    
    # 그룹 내 문장이 2개 미만이면 계산할 수 없으므로 0 또는 np.nan 처리
    if len(sentences) < 2:
        avg_similarities[category] = np.nan
        continue
    
    embeddings = model.encode(sentences, convert_to_tensor=False)
    
    sim_matrix = cosine_similarity(embeddings)
    
    # 대각선은 자기 자신과의 유사도로 제외 (k=1부터 시작)
    n = sim_matrix.shape[0]
    upper_tri_indices = np.triu_indices(n, k=1)
    pairwise_sims = sim_matrix[upper_tri_indices]
    
    avg_similarity = np.mean(pairwise_sims)
    avg_similarities[category] = avg_similarity

# 결과 DataFrame 생성
result_df = pd.DataFrame(list(avg_similarities.items()), columns=['공종(중분류)', '평균유사도'])
print(result_df)

         공종(중분류)     평균유사도
0           가설공사  0.546390
1         강구조물공사  0.545715
2         건축 토공사  0.539231
3       건축물 부대공사  0.530162
4            관공사  0.542984
5       관공사 부대공사  0.538104
6           교량공사  0.539067
7           금속공사  0.538962
8         기계설비공사  0.493029
9             기타  0.512591
10      댐 및 제방공사  0.525409
11     도로 및 포장공사  0.551911
12          도장공사  0.556861
13          말뚝공사  0.555806
14           목공사  0.544218
15          미장공사  0.547937
16          방수공사  0.536368
17        산업설비공사  0.442414
18          수장공사  0.539619
19        전기설비공사  0.481451
20          조경공사  0.556957
21          조적공사  0.544845
22        지반개량공사  0.542257
23          지반조사  0.523460
24     지붕 및 홈통공사  0.502272
25          지정공사  0.547190
26     창호 및 유리공사  0.530560
27          철골공사  0.531484
28      철근콘크리트공사  0.542627
29     철도 및 궤도공사  0.533192
30      타일 및 돌공사  0.535059
31          터널공사  0.499065
32           토공사  0.526639
33        통신설비공사  0.577286
34      특수 건축물공사  0.538375
35  프리캐스트 콘크리트공사  0.500266
3

In [15]:
overall_avg = np.nanmean(list(avg_similarities.values()))
print("전체 평균 유사도:", overall_avg)

전체 평균 유사도: 0.532339


In [16]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

grouped = df.groupby("공사종류(대분류)")
# 결과 저장을 위한 딕셔너리
avg_similarities = {}

# 각 그룹에 대해 평균 코사인 유사도 계산
for category, group in grouped:
    sentences = group['재발방지대책 및 향후조치계획'].tolist()
    
    # 그룹 내 문장이 2개 미만이면 계산할 수 없으므로 0 또는 np.nan 처리
    if len(sentences) < 2:
        avg_similarities[category] = np.nan
        continue
    
    embeddings = model.encode(sentences, convert_to_tensor=False)
    
    sim_matrix = cosine_similarity(embeddings)
    
    # 대각선은 자기 자신과의 유사도로 제외 (k=1부터 시작)
    n = sim_matrix.shape[0]
    upper_tri_indices = np.triu_indices(n, k=1)
    pairwise_sims = sim_matrix[upper_tri_indices]
    
    avg_similarity = np.mean(pairwise_sims)
    avg_similarities[category] = avg_similarity

# 결과 DataFrame 생성
result_df = pd.DataFrame(list(avg_similarities.items()), columns=['공사종류(대분류)', '평균유사도'])
print(result_df)

  공사종류(대분류)     평균유사도
0        건축  0.531881
1    산업환경설비  0.496907
2        조경  0.556933
3        토목  0.541222


In [17]:
overall_avg = np.nanmean(list(avg_similarities.values()))
print("전체 평균 유사도:", overall_avg)

전체 평균 유사도: 0.53173584
